In [0]:
from keras.layers import Input,LSTM,Bidirectional,Concatenate,Dot, Multiply
from keras.layers import RepeatVector, Dense, Activation
from keras.optimizers import Adam
from keras.models import load_model, Model
from keras.activations import softmax
import keras.backend as K
import numpy as np
import re

In [0]:
def read_data(path,seq_question,seq_answer):
    with open(path) as load_file:
        lines = load_file.readlines()
        for line in lines:
            question,answer = line.strip("\n").split("@@@@@")
            seq_question.append(question)
            seq_answer.append(answer)
    

In [0]:
#nhập các câu input output vào mảng
sequence_input = []
sequence_target = []
read_data("1_qa.txt",sequence_input,sequence_target)
read_data("2_qa.txt",sequence_input,sequence_target)
read_data("3_qa.txt",sequence_input,sequence_target)


In [0]:
#tính dộ dài Tx: độ lớn đầu vào, Ty độ lớn chuỗi đầu ra
Tx = max([len(line.split()) for line in sequence_input])
Ty = max([len(line.split()) for line in sequence_target])

In [0]:
#loại các kí tự thừa
for index,_ in enumerate(sequence_input):
    sequence_input[index] = re.sub(r'[?)(,-/\"]',"",sequence_input[index])
    sequence_target[index] = re.sub(r'[?.)(,-/\"]',"",sequence_target[index])


In [0]:
#tính số từ trong chuôi
input_word = set()             #lưu các từ vụng trong câu vào
target_word = set()            #lưu các từ vựng trong câu ra
for i in range(len(sequence_input)):
    word_input = sequence_input[i].split()
    for word in word_input:
        if word.lower() not in input_word:
            input_word.add(word.lower())
    word_target = sequence_target[i].split()
    for word in word_target:
        if word.lower() not in target_word:
            target_word.add(word.lower())
#sắp xếp theo thứ tự
input_word = sorted(input_word)
target_word= sorted(target_word)

In [0]:
#tạo 1 dict chứa từ và vị trí của nó trong tập từ vựng
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_word)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_word)])


In [0]:
#tạo mảng biểu diễn câu input,output
encoder_input_data = np.zeros(
    [len(sequence_input),Tx,len(input_word)],dtype = float)
encoder_output_data = np.zeros(
    [len(sequence_target),Ty,len(target_word)],dtype=float)

In [0]:
#chuyển các câu đầu vào thành ma trận biểu diễn
for i in range(len(sequence_input)):
    for t,input_text in enumerate(sequence_input[i].lower().split()):
        index = input_token_index[input_text]
        encoder_input_data[i][t][index] = 1
for i in range(len(sequence_input)):
    for t,output_text in enumerate(sequence_target[i].lower().split()):
        index = target_token_index[output_text]
        encoder_output_data[i][t][index] = 1

In [0]:
def softmax(x, axis=1):
    """Softmax activation function.
    # Arguments
        x : Tensor.
        axis: Integer, axis along which the softmax normalization is applied.
    # Returns
        Tensor, output of softmax transformation.
    # Raises
        ValueError: In case `dim(x) == 1`.
    """
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')
        

In [0]:
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [0]:
def one_step_attention(a, s_prev):
    """
    a: vecto của Bidirectional
    s_prev: trạng thái trước đó
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attetion) LSTM cell
    """
    
   
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    concat = concatenator([a, s_prev])
    # Use densor to propagate concat through a small fully-connected neural network to compute the "energies" variable e. (≈1 lines)
    e = densor(concat)
    # Use activator and e to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(e) # We are using a custom softmax(axis = 1) loaded in this notebook
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas, a])
    
    
    return context

In [0]:
n_a = 64
n_s = 128
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(target_word), activation=softmax)

In [0]:
# GRADED FUNCTION: model

def model(Tx, Ty, n_a, n_s, input_vocab_size, output_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 and c0, initial hidden state for the decoder LSTM of shape (n_s,)
    X = Input(shape=(Tx,input_vocab_size)) 
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    
    
    # Step 1: Define your pre-attention Bi-LSTM. Remember to use return_sequences=True. (≈ 1 line)
    a = Bidirectional(LSTM(n_a, return_sequences=True))(X)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(a, s)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model([X, s0, c0], outputs)
    
    
    return model

In [20]:
model = model(Tx, Ty, n_a, n_s, len(input_word), len(target_word))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 124, 1393)    0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 128)          0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 124, 128)     746496      input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 124, 128)     0           s0[0][0]                         
                                                                 lstm_1[0][0]                     
          

In [0]:

opt = Adam(lr = 0.005, beta_1=0.9, beta_2=0.999, decay = 0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


In [0]:
s0 = np.zeros((len(sequence_input), n_s))
c0 = np.zeros((len(sequence_input), n_s))
outputs =list(encoder_output_data.swapaxes(0,1))


In [23]:
encoder_input_data.shape

(777, 124, 1393)

In [24]:
model.fit([encoder_input_data, s0, c0],outputs, epochs=30, batch_size=10)

Epoch 1/30
777/777 [==============================] - 224s 289ms/step - loss: 37.4410 - dense_2_loss: 0.0072 - dense_2_acc: 0.1094 - dense_2_acc_1: 0.0193 - dense_2_acc_2: 0.0746 - dense_2_acc_3: 0.0142 - dense_2_acc_4: 0.0450 - dense_2_acc_5: 0.0116 - dense_2_acc_6: 0.0154 - dense_2_acc_7: 0.0077 - dense_2_acc_8: 0.0103 - dense_2_acc_9: 0.0077 - dense_2_acc_10: 0.0064 - dense_2_acc_11: 0.0077 - dense_2_acc_12: 0.0051 - dense_2_acc_13: 0.0013 - dense_2_acc_14: 0.0051 - dense_2_acc_15: 0.0000e+00 - dense_2_acc_16: 0.0013 - dense_2_acc_17: 0.0013 - dense_2_acc_18: 0.0051 - dense_2_acc_19: 0.0064 - dense_2_acc_20: 0.0039 - dense_2_acc_21: 0.0026 - dense_2_acc_22: 0.0051 - dense_2_acc_23: 0.0000e+00 - dense_2_acc_24: 0.0051 - dense_2_acc_25: 0.0000e+00 - dense_2_acc_26: 0.0000e+00 - dense_2_acc_27: 0.0000e+00 - dense_2_acc_28: 0.0026 - dense_2_acc_29: 0.0039 - dense_2_acc_30: 0.0000e+00 - dense_2_acc_31: 0.0000e+00 - dense_2_acc_32: 0.0000e+00 - dense_2_acc_33: 0.0000e+00 - dense_2_acc_34:

In [0]:
model.save_weights('model.h5')

In [0]:
model.load_weights('model.h5')

In [0]:
target1_token_index = dict(
    [(i,word) for i, word in enumerate(target_word)])


In [0]:
source = encoder_input_data[10]
# source = source.T
predict = model.predict([[source,],s0,c0])
prediction = np.argmax(predict, axis = -1)


In [31]:
prediction.shape

(104, 1)

In [30]:
#30
output = ""
for i in prediction.reshape(Ty):
    output = output+" "+target1_token_index[i]
print(output)    

 gs điểm viên viên viên viên và và sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh sinh
